In [1]:
%%capture
import warnings

import numpy as np
import pandas as pd
import streamlit as st
from likelihood.tools import *
from tensorflow.keras.models import load_model

warnings.filterwarnings("ignore")
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import matplotlib.pyplot as plt
import tensorflow as tf

plt.rcParams["font.size"] = 12
plt.style.use("bmh")

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from figure import *

# This files are in the forecasting folder
from series import *

np.random.seed(0)
neural_network = load_model("./models/model_tensor")
# If necessary, we compile manually
neural_network.compile(loss="mse", optimizer="sgd", metrics=["mae"])

In [2]:
# The dataset is generated
# The window of your sets used in pre-training should be similar to the one used in retraining ("n_steps").
# For the previous step, check model_tensor.py
steps = 16
data_ = generate_series(n=25, n_steps=100, incline=False)
scaler = DataScaler(np.copy(data_))
data_scale = scaler.rescale()
# Data pre-processing
series = convert_to_df(np.copy(data_)).describe()
train, test = create_train_test_set(np.copy(data_scale), p_train=0.8, steps=steps)
# Retraining of the pre-training model
neural_network.fit(train[0][:, :, np.newaxis], train[1], verbose=0)
prediction = forecasting(
    neural_network, train[0][:, :, np.newaxis], data_.shape[1], scaler, steps=steps
)

In [3]:
plot_series(convert_to_df(data_), prediction, i=0)